In [18]:
from automated_statistician import greedy_search
import GPy
import numpy as np
import matplotlib.pyplot as plt
import utils
from functools import partial
from utils import KernelFunction, KernelEnvironment, log_likelihood_reward
from utils import plot_kernel_function, compare_kernels
from gflownet import GFlowNet 
import torch.nn.functional as F
from torch.distributions import Categorical
import torch
from utils import ForwardPolicy, BackwardPolicy
import random
from utils import train

import evaluation
from evaluation import create_random_kernel

In [19]:
ll = -1
while ll < 0:
    true_kernel = create_random_kernel()
    X, Y, true_kernel_str = utils.generate_gp_data(true_kernel, input_dim=1, n_points=30, noise_var=1e-4)
    X_test, Y_test, _ = utils.generate_gp_data(true_kernel, input_dim=1, n_points=30, noise_var=1e-4)
    ll = utils.evaluate_likelihood(true_kernel, X, Y, runtime=False)

print("True Kernel:", true_kernel_str, "Log Marginal Likelihood:", utils.evaluate_likelihood(true_kernel, X, Y, runtime=False))
print("True Kernel:", true_kernel_str, "Log Marginal Likelihood:", utils.evaluate_likelihood(true_kernel, X_test, Y_test, runtime=False))

  -> Randomizing 'RBF' params...
  -> Randomizing 'RBF' params...
True Kernel: (RBF({'lengthscale': 0.924, 'variance': 1.2}) + RBF({'lengthscale': 1.234, 'variance': 1.102})) Log Marginal Likelihood: 40.55135800840988
True Kernel: (RBF({'lengthscale': 0.924, 'variance': 1.2}) + RBF({'lengthscale': 1.234, 'variance': 1.102})) Log Marginal Likelihood: 37.91051503574889


In [20]:
greedy_kernel, _ = greedy_search(X, Y, method='BIC', max_steps=5)

[Step 1] BIC: -67.97 LL: 39.09 | RBF({'lengthscale': 1.0, 'variance': 1.0})
[Step 2] BIC: -64.57 LL: 39.09 | (RBF({'lengthscale': 1.0, 'variance': 1.0}) * Constant({'variance': 1.0}))
[Step 3] BIC: -67.97 LL: 39.09 | RBF({'lengthscale': 1.0, 'variance': 1.0})
[Step 4] BIC: -64.57 LL: 39.09 | (RBF({'lengthscale': 1.0, 'variance': 1.0}) * Constant({'variance': 1.0}))
[Step 5] BIC: -67.97 LL: 39.09 | RBF({'lengthscale': 1.0, 'variance': 1.0})


In [21]:
ll_test = utils.evaluate_likelihood(true_kernel, X_test, Y_test, runtime=False)
rmse_test = evaluation.calculate_rmse(true_kernel, X, Y, X_test, Y_test)
    
print("True kernel")
print(" Log Marginal Likelihood:", ll_test, "RMSE:", rmse_test)
print("===========================================")


ll_test = utils.evaluate_likelihood(greedy_kernel, X_test, Y_test, runtime=False)
rmse_test = evaluation.calculate_rmse(greedy_kernel, X, Y, X_test, Y_test)
    
print("Greedy Kernel (automated statistician)")
print("Log Marginal Likelihood:", ll_test, "RMSE:", rmse_test)
print("===========================================")

rbf_kernel = KernelFunction().rbf()
linear_kernel = KernelFunction().linear()
periodic_kernel = KernelFunction().periodic()
constant_kernel = KernelFunction().constant()

kernels = [rbf_kernel, linear_kernel, periodic_kernel, constant_kernel]
for kernel in kernels:
    ll_test = utils.evaluate_likelihood(kernel, X_test, Y_test, runtime=False)
    rmse_test = evaluation.calculate_rmse(kernel, X, Y, X_test, Y_test)
    
    print(f"{kernel.name} Kernel")
    print("Log Marginal Likelihood:", ll_test, "RMSE:", rmse_test)
print("===========================================")



True kernel
 Log Marginal Likelihood: 37.91051503574889 RMSE: 0.46183904401792775
Greedy Kernel (automated statistician)
Log Marginal Likelihood: 37.91051503574819 RMSE: 0.43624261807136194
RBF Kernel
Log Marginal Likelihood: 37.91051503574819 RMSE: 0.43624261807136194
Linear Kernel
Log Marginal Likelihood: -16.97927533525597 RMSE: 0.4153390353624188
Periodic Kernel
Log Marginal Likelihood: -9.553237715380403 RMSE: 0.387475342223833
Constant Kernel
Log Marginal Likelihood: -10.543301305622878 RMSE: 0.3779038877274488
